import the necessary library

In [1]:
import pandas as pd
# !pip install psycopg2
import pandas as pd
import psycopg2
from psycopg2 import Error
from sqlalchemy import create_engine


In [2]:
# Provide the file paths to the downloaded CSV files
borrower_csv_path = 'Borrower_Data - Sheet1.csv'
loan_csv_path = 'Loan_Data - Sheet1 - Loan_Data - Sheet1.csv'
payment_schedule_csv_path = 'Schedule_Data.xlsx - Sheet1.csv'
repayment_csv_path = 'Repayment_Data - Sheet1 - Repayment_Data - Sheet1 (1).csv'

# Read the CSV files into pandas DataFrames
borrower_df = pd.read_csv(borrower_csv_path)
loan_df = pd.read_csv(loan_csv_path)
payment_schedule_df = pd.read_csv(payment_schedule_csv_path)
repayment_df = pd.read_csv(repayment_csv_path)


In [3]:
borrower_df.head()

,Borrower_Id,State,City,zip code,borrower_credit_score
0,123fd35,dfgc,12olki,19473,4
1,123fd36,rtgio,89hfg,923475,a


In [4]:
loan_df.head()

,Borrower_id,loan_id,Date_of_release,Term,InterestRate,LoanAmount,Downpayment,Payment_frequency,Maturity_date
0,123fd35,32u09wekjbfje,01/27/2021,12,1.05,1209484,124993,1.007903e+05,01/31/2022
1,123fd36,9190i0-nbfb,02/01/2021,24,1.05,12394031,1000000,5.164180e+05,02/29/2023
2,123fd35,09u924rbwf,04/16/2021,36,1.05,45784937,1200000,1.271804e+06,04/30/2024
3,123fd36,3240-9rfwb,03/26/2021,12,1.05,1234890,132000,1.029075e+05,03/31/2022


In [5]:
payment_schedule_df.head()

,loan_id,schedule_id,Expected_payment_date,Expected_payment_amount
0,32u09wekjbfje,3434r409km123456,02/27/2021,100790.3333
1,32u09wekjbfje,3434r409km123457,03/27/2021,100790.3333
2,32u09wekjbfje,3434r409km123458,04/27/2021,100790.3333
3,32u09wekjbfje,3434r409km123459,05/27/2021,100790.3333
4,32u09wekjbfje,3434r409km123460,06/27/2021,100790.3333


In [6]:
repayment_df.head()

,loan_id(fk),payment_id(pk),Amount_paid,Date_paid
0,32u09wekjbfje,3434r409kmPAID123456,03/31/2021,100790.3333
1,32u09wekjbfje,3434r409kmPAID123457,03/31/2021,100790.3333
2,32u09wekjbfje,3434r409kmPAID123458,04/27/2021,100790.3333
3,32u09wekjbfje,3434r409kmPAID123459,05/27/2021,100790.3333
4,32u09wekjbfje,3434r409kmPAID123460,06/27/2021,100790.3333


In [7]:
repayment_df['Amount_paid'] = pd.to_datetime(repayment_df['Amount_paid'], errors='coerce')


In [8]:
repayment_df.head()

,loan_id(fk),payment_id(pk),Amount_paid,Date_paid
0,32u09wekjbfje,3434r409kmPAID123456,2021-03-31,100790.3333
1,32u09wekjbfje,3434r409kmPAID123457,2021-03-31,100790.3333
2,32u09wekjbfje,3434r409kmPAID123458,2021-04-27,100790.3333
3,32u09wekjbfje,3434r409kmPAID123459,2021-05-27,100790.3333
4,32u09wekjbfje,3434r409kmPAID123460,2021-06-27,100790.3333


In [9]:
loan_df['Date_of_release'] = pd.to_datetime(loan_df['Date_of_release'], errors='coerce')
loan_df['Maturity_date'] = pd.to_datetime(loan_df['Maturity_date'], errors='coerce')

In [10]:
loan_df.head()

,Borrower_id,loan_id,Date_of_release,Term,InterestRate,LoanAmount,Downpayment,Payment_frequency,Maturity_date
0,123fd35,32u09wekjbfje,2021-01-27,12,1.05,1209484,124993,1.007903e+05,2022-01-31
1,123fd36,9190i0-nbfb,2021-02-01,24,1.05,12394031,1000000,5.164180e+05,NaT
2,123fd35,09u924rbwf,2021-04-16,36,1.05,45784937,1200000,1.271804e+06,2024-04-30
3,123fd36,3240-9rfwb,2021-03-26,12,1.05,1234890,132000,1.029075e+05,2022-03-31


In [11]:
payment_schedule_df['Expected_payment_date'] = pd.to_datetime(payment_schedule_df['Expected_payment_date'], errors='coerce')

In [12]:
payment_schedule_df.head()

,loan_id,schedule_id,Expected_payment_date,Expected_payment_amount
0,32u09wekjbfje,3434r409km123456,2021-02-27,100790.3333
1,32u09wekjbfje,3434r409km123457,2021-03-27,100790.3333
2,32u09wekjbfje,3434r409km123458,2021-04-27,100790.3333
3,32u09wekjbfje,3434r409km123459,2021-05-27,100790.3333
4,32u09wekjbfje,3434r409km123460,2021-06-27,100790.3333


In [13]:
repayment_df.rename(columns={'Amount_paid':'payment_date'},inplace=True)
repayment_df.rename(columns={'Date_paid':'Amount_paid'},inplace=True)

In [14]:
repayment_df.head()

,loan_id(fk),payment_id(pk),payment_date,Amount_paid
0,32u09wekjbfje,3434r409kmPAID123456,2021-03-31,100790.3333
1,32u09wekjbfje,3434r409kmPAID123457,2021-03-31,100790.3333
2,32u09wekjbfje,3434r409kmPAID123458,2021-04-27,100790.3333
3,32u09wekjbfje,3434r409kmPAID123459,2021-05-27,100790.3333
4,32u09wekjbfje,3434r409kmPAID123460,2021-06-27,100790.3333


In [15]:
borrower_df.rename(columns={'zip code':'zip_code'},inplace=True)

In [16]:
borrower_df.head()

,Borrower_Id,State,City,zip_code,borrower_credit_score
0,123fd35,dfgc,12olki,19473,4
1,123fd36,rtgio,89hfg,923475,a


In [17]:
try:
    # Replace the connection details with your own
    connection = psycopg2.connect(
        host="localhost",
        database="autochek_db",
        user="auto",
        password="auto"
    )
    cursor = connection.cursor()
    print("Successfully connected to the PostgreSQL database")

except (Exception, Error) as e:
    print("Error while connecting to the PostgreSQL database:", e)


Successfully connected to the PostgreSQL database


In [18]:
loan_df.head()

,Borrower_id,loan_id,Date_of_release,Term,InterestRate,LoanAmount,Downpayment,Payment_frequency,Maturity_date
0,123fd35,32u09wekjbfje,2021-01-27,12,1.05,1209484,124993,1.007903e+05,2022-01-31
1,123fd36,9190i0-nbfb,2021-02-01,24,1.05,12394031,1000000,5.164180e+05,NaT
2,123fd35,09u924rbwf,2021-04-16,36,1.05,45784937,1200000,1.271804e+06,2024-04-30
3,123fd36,3240-9rfwb,2021-03-26,12,1.05,1234890,132000,1.029075e+05,2022-03-31


In [19]:
# SQL statements to create the tables
create_table_queries = [
    '''
    CREATE TABLE IF NOT EXISTS Borrower_table (
        Borrower_id INTEGER PRIMARY KEY,
        State TEXT,
        City TEXT,
        zip_code INTEGER,
        borrower_credit_score INTEGER
    )
    ''',
    '''
    CREATE TABLE IF NOT EXISTS Loan_table (
        Borrower_id INTEGER REFERENCES Borrower_table (Borrower_id),
        loan_id INTEGER PRIMARY KEY,
        Date_of_release DATE,
        Term INTEGER,
        InterestRate FLOAT,
        LoanAmount FLOAT,
        Downpayment FLOAT,
        Payment_frequency TEXT,
        Maturity_date DATE
    )
    
    ''' ,
    '''
    CREATE TABLE IF NOT EXISTS Schedule_Table (
        loan_id INTEGER REFERENCES Loan_table (Loan_id),
        schedule_id INTEGER PRIMARY KEY,
        Expected_payment_date DATE,
        Expected_payment_amount FLOAT
    )
    
    ''' ,
    '''
    CREATE TABLE IF NOT EXISTS Repayment_Table (
        loan_id INTEGER REFERENCES Loan_table (Loan_id),
        payment_id INTEGER PRIMARY KEY,
        payment_date DATE,
        Amount_paid FLOAT
    )
    '''

]
try:
    # Execute the create table queries
    for query in create_table_queries:
        cursor.execute(query)
    connection.commit()
    print("Tables created successfully")

except (Exception, Error) as e:
    print("Error while creating tables:", e)

Tables created successfully


In [20]:
# Create a SQLAlchemy engine using the connection
engine = create_engine('postgresql+psycopg2://', creator=lambda: connection)

# Use the engine to load data into tables
try:
    
    borrower_df.to_sql('Borrower_table', con=engine, if_exists='replace', index=False)
    loan_df.to_sql('Loan_table', con=engine, if_exists='replace', index=False)
    payment_schedule_df.to_sql('Schedule_Table', con=engine, if_exists='replace', index=False)
    repayment_df.to_sql('Repayment_Table', con=engine, if_exists='replace', index=False)
    
    print("Data loaded into the tables successfully")

except (Exception, Error) as e:
    print("Error while loading data into tables:", e)

Data loaded into the tables successfully


Error while executing the SQL query: function regexp_replace(integer, unknown, unknown) does not exist
LINE 37:     Schedule_table p ON p.loan_id = r.loan_id AND REGEXP_REP...
                                                           ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.

